In [3]:
import project_path

import pandas as pd
import numpy as np
from pathlib import Path
import datetime

import brightway2 as bw
import wurst

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.lines import Line2D
from matplotlib.patches import Patch
import matplotlib.cm as cm
import geopandas as gpd
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import seaborn as sns

from src import inventory_imports
from src import results_analysis

pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [8]:
# To be changed accordingly:
BW_PROJECT = 'iri_work'                          # name of your project
bw.projects.set_current(BW_PROJECT)              # set project

ECOINVENT_DB = 'ecoinvent 3.9.1 cutoff'          # name of ecoinvent database in your project
LCI_DB = 'biomethane-to-ammonia'                 # name of LCIs database

DATA_DIR = Path("../data")
FIG_EXPORT_DIR = Path("../reports/figures")


In [3]:
LCIA_METHODS = {
                'Climate change, GWP 100a':            ('IPCC 2021', 'climate change', 'GWP 100a, incl. H and bio CO2'),
                'Acidification':                       ('EF v3.1', 'acidification', 'accumulated exceedance (AE)'),
                'Eutrophication, freshwater':          ('EF v3.1', 'eutrophication: freshwater', 'fraction of nutrients reaching freshwater end compartment (P)'),
                'Eutrophication, marine':              ('EF v3.1', 'eutrophication: marine', 'fraction of nutrients reaching marine end compartment (N)'),
                'Eutrophication, terrestrial':         ('EF v3.1', 'eutrophication: terrestrial', 'accumulated exceedance (AE)'),
                'Photochemical oxidant formation':     ('EF v3.1', 'photochemical oxidant formation: human health', 'tropospheric ozone concentration increase'),
                'Particulate matter formation':        ('EF v3.1', 'particulate matter formation', 'impact on human health'),
                'Ozone depletion':                     ('EF v3.1', 'ozone depletion', 'ozone depletion potential (ODP)'),
                'Ecotoxicity, freshwater':             ('EF v3.1', 'ecotoxicity: freshwater', 'comparative toxic unit for ecosystems (CTUe)'),
                'Human toxicity, carcinogenic':        ('EF v3.1', 'human toxicity: carcinogenic', 'comparative toxic unit for human (CTUh)'),
                'Human toxicity, non-carcinogenic':    ('EF v3.1', 'human toxicity: non-carcinogenic', 'comparative toxic unit for human (CTUh)'),
                'Ionising radiation':                  ('EF v3.1', 'ionising radiation: human health', 'human exposure efficiency relative to u235'), 
                'Energy resources, non-renewable':     ('EF v3.1', 'energy resources: non-renewable', 'abiotic depletion potential (ADP): fossil fuels'),
                'Material resources, minerals/metals': ('EF v3.1', 'material resources: metals/minerals', 'abiotic depletion potential (ADP): elements (ultimate reserves)'),
                'Land use':                            ('EF v3.1', 'land use', 'soil quality index'),
                'Water use':                           ('EF v3.1', 'water use', 'user deprivation potential (deprivation-weighted water consumption)')
               }

IPCC_METHOD = {k: v for (k,v) in LCIA_METHODS.items() if 'Climate change' in k}

In [4]:
INVENTORIES = {
               'Natural gas':                                            ('ammonia production, liquid, fossil ammonia from natural gas', 'ammonia, anhydrous, liquid'),
               'Natural gas + CCS Syngas':                               ('ammonia production, liquid, fossil ammonia from natural gas, syngas w/ CCS', 'ammonia, anhydrous, liquid'),
               'Natural gas + CCS Syngas + CCS Heating':                 ('ammonia production, liquid, fossil ammonia from natural gas, syngas w/ CCS, heating w/ CCS', 'ammonia, anhydrous, liquid'),
               'Green H2':                                               ('ammonia production, liquid, green ammonia from wind-based hydrogen', 'ammonia, anhydrous, liquid'),
               'Biomethane':                                             ('ammonia production, liquid, bio-ammonia from biomethane', 'ammonia, anhydrous, liquid'),
               'Biomethane + CCS Syngas':                                ('ammonia production, liquid, bio-ammonia from biomethane, syngas w/ CCS', 'ammonia, anhydrous, liquid'),
               'Biomethane + CCS Syngas + CCS Heating':                  ('ammonia production, liquid, bio-ammonia from biomethane, syngas w/ CCS, heating w/ CCS', 'ammonia, anhydrous, liquid'),
               'Biomethane + CCS Upgrading':                             ('ammonia production, liquid, bio-ammonia from biomethane w/ CCS', 'ammonia, anhydrous, liquid'),
               'Biomethane + CCS Upgrading + CCS Syngas':                ('ammonia production, liquid, bio-ammonia from biomethane w/ CCS, syngas w/ CCS', 'ammonia, anhydrous, liquid'),
               'Biomethane + CCS Upgrading + CCS Syngas + CCS Heating':  ('ammonia production, liquid, bio-ammonia from biomethane w/ CCS, syngas w/ CCS, heating w/ CCS', 'ammonia, anhydrous, liquid'),
               }

In [1]:
biomethane_fossil_match = {'Biomethane': 'Natural gas',
                           'Biomethane + CCS Syngas': 'Natural gas + CCS Syngas',
                           'Biomethane + CCS Syngas + CCS Heating': 'Natural gas + CCS Syngas + CCS Heating',
                           'Biomethane + CCS Upgrading': 'Natural gas',
                           'Biomethane + CCS Upgrading + CCS Syngas': 'Natural gas + CCS Syngas',
                           'Biomethane + CCS Upgrading + CCS Syngas + CCS Heating': 'Natural gas + CCS Syngas + CCS Heating'
                          }